In [4]:
%load_ext autoreload
%autoreload 2
from stresstest.util import load_json
import os
from loguru import logger
#logger.remove(0)
print(os.getcwd())
predictions_bidaf = load_json('../data/modifier/bidaf/predictions-bidaf-ssq-modifier-retrieval-50-5.json')
gold = load_json('../data/modifier/stresstest-ssq-modifier-retrieval-50-5.json')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/viktor/workspace/mrc-stresstest/notebooks


Sort by reasoning

In [2]:
from collections import defaultdict
by_reasoning = []
for sample, prediction in zip(gold, predictions_bidaf):
    by_reasoning_by_sample = defaultdict(list)
    for item in sample:
        passage_id, passage, qas = item['id'], item['passage'], item['qas']
        for qa in qas:
            by_reasoning_by_sample[qa['reasoning']].append((
                passage_id,
                passage,
                qa['id'],
                qa['question'],
                qa['answer']
            ))
    by_reasoning.append(by_reasoning_by_sample)

In [7]:
from stresstest.eval_utils import em, f1, get_mean_var_ci
for reasoning_type in by_reasoning[0].keys():
    print(f"REASONING TYPE: {reasoning_type}")
    per_type_em = []
    per_type_f1 = []
    for i, sample in enumerate(by_reasoning):
        per_type_em.append(em(sample[reasoning_type], predictions_bidaf[i]))
        per_type_f1.append(f1(sample[reasoning_type], predictions_bidaf[i]))
    print("EM", get_mean_var_ci(per_type_em))
    print("F1", get_mean_var_ci(per_type_f1))
for sample in by_reasoning:
    for reasoning_type, examples in sample.items():
        print(f"{reasoning_type}: {len(examples)}")

REASONING TYPE: retrieval
EM (0.5842751190520398, 0.0009631531261842735, 0.038534680129260035)
F1 (0.6379551232392233, 0.0005347085551407633, 0.028711947775104826)
REASONING TYPE: ordering-easy
EM (0.23454001979599437, 0.000835246520801456, 0.03588487408214859)
F1 (0.2641685476960889, 0.0009379222289829009, 0.0380266006175976)
retrieval: 207
ordering-easy: 272
retrieval: 197
ordering-easy: 270
retrieval: 230
ordering-easy: 266
retrieval: 223
ordering-easy: 277
retrieval: 208
ordering-easy: 289


In [8]:
from collections import defaultdict
by_question_type = []
for sample, prediction in zip(gold, predictions_bidaf):
    by_question_type_by_sample = defaultdict(list)
    for item in sample:
        passage_id, passage, qas = item['id'], item['passage'], item['qas']
        for qa in qas:
            by_question_type_by_sample[qa['type']].append((
                passage_id,
                passage,
                qa['id'],
                qa['question'],
                qa['answer']
            ))
    by_question_type.append(by_question_type_by_sample)

In [9]:

from stresstest.eval_utils import em, f1, get_mean_var_ci
for question_type in by_question_type[0].keys():
    print(f"QUESTION TYPE: {question_type}")
    per_type_em = []
    per_type_f1 = []
    for i, sample in enumerate(by_question_type):
        per_type_em.append(em(sample[question_type], predictions_bidaf[i]))
        per_type_f1.append(f1(sample[question_type], predictions_bidaf[i]))
    print("EM", get_mean_var_ci(per_type_em))
    print("F1", get_mean_var_ci(per_type_f1))

for sample in by_question_type:
    for question_type, examples in sample.items():
        print(f"{question_type}: {len(examples)}")

QUESTION TYPE: direct
EM (0.36219364805333026, 0.0008418008236389397, 0.03602539590103246)
F1 (0.40086195229098387, 0.0007186728563079428, 0.03328662091942248)
QUESTION TYPE: overall
EM (0.7300861649248747, 0.0042694585734576055, 0.08113168410229714)
F1 (0.7878858987352361, 0.005417155671379586, 0.09138812078351405)
direct: 447
overall: 32
direct: 445
overall: 22
direct: 440
overall: 56
direct: 462
overall: 38
direct: 463
overall: 34


EM misses for retrieval

In [6]:
from stresstest.util import  sample_iter
for sample, predictions in zip(gold, predictions_bidaf):
    for story_id, story, question_id, question, answer, qa in sample_iter(sample, True):
        answer = answer or ''
        prediction = predictions[story_id][question_id]
        if "".join(str(answer).lower().split()) != "".join(str(prediction).lower().split()) and not 'ordering' in qa['reasoning']:
            print(20*"====")
            print("Question:", question)
            print("Passage: ", story)
            print("Answer:", answer)
            print("Prediction:", prediction)

Question: Who committed the 1 th foul?
Passage:  The match started as Elsie Vajgrt scored on the 67 th minute scoring the ball in the right corner after she ran 14 metres . But FC Duckburg retaliated as Laura Owens slotted in a goal for FC Duckburg . Nina Green , one of Arctic Whalesharks 's better performers today, constituted a counter strike as she scored on the 87 th minute and for her 10 th league goal of the season it was spectacular . Nina Green scored on the 88 th minute to continue where Arctic Whalesharks left off when she dribbled 2 metres on the flank and intercepted FC Duckburg 's goalkeeper's goal kick before drilling in the ball off the post and in the back of the net . Things proceeded with Elsie Vajgrt disappointing the crowd with an unnecessary action , felling Laura Owens near the 6-yard-area .
Answer: Elsie Vajgrt
Prediction: Elsie Vajgrt disappointing the crowd with an unnecessary action , felling Laura Owens
Question: Who fouled?
Passage:  The match started as Els

EM misses for overall questions

In [10]:
from stresstest.util import  sample_iter
for sample, predictions in zip(gold, predictions_bidaf):
    for story_id, story, question_id, question, answer, qa in sample_iter(sample, True):
        answer = answer or ''
        prediction = predictions[story_id][question_id]
        if "".join(str(answer).lower().split()) != "".join(str(prediction).lower().split()) and qa['type'] == 'overall':
            print(20*"====")
            print("Question:", question)
            print("Passage: ", story)
            print("Answer:", answer)
            print("Prediction:", prediction)


Question: Who fouled?
Passage:  The match started as Elsie Vajgrt scored on the 67 th minute scoring the ball in the right corner after she ran 14 metres . But FC Duckburg retaliated as Laura Owens slotted in a goal for FC Duckburg . Nina Green , one of Arctic Whalesharks 's better performers today, constituted a counter strike as she scored on the 87 th minute and for her 10 th league goal of the season it was spectacular . Nina Green scored on the 88 th minute to continue where Arctic Whalesharks left off when she dribbled 2 metres on the flank and intercepted FC Duckburg 's goalkeeper's goal kick before drilling in the ball off the post and in the back of the net . Things proceeded with Elsie Vajgrt disappointing the crowd with an unnecessary action , felling Laura Owens near the 6-yard-area .
Answer: Elsie Vajgrt
Prediction: Laura Owens
Question: Who fouled?
Passage:  The match started as Carolyne Goodrich hammered in a goal for Arctic Duckburg . The onslaught continued, as Phyllis

Modified only:

In [16]:
import textwrap
from stresstest.util import  sample_iter
for sample, predictions in zip(gold, predictions_bidaf):
    ctr = 0
    for story_id, story, question_id, question, answer, qa in sample_iter(sample, True):
        if qa['question_data']['modified'] and not qa['question_data'].get('easier', False):
            ctr += 1
            answer = answer or ''
            prediction = predictions[story_id][question_id]
            #if "".join(str(answer).lower().split()) != "".join(str(prediction).lower().split()) and qa['type'] == 'overall':
            print(20*"====")
            print("Question ID:", question_id)
            print("Question:", question)
            print("Passage: ")
            print(textwrap.fill(story, width=80))
            print("Answer:", answer)
            print("Prediction:", prediction)




Question ID: 543759e673031ef382fd8fb3cab786ab
Question: Who scored the 1 th goal?
Passage: 
A spectacular 23 metres strike from Elsie Vajgrt nearly homing into the lower
corner past the last line of defence for her 5 th league goal of the season
opened the action . But FC Duckburg retaliated as Laura Owens nearly hammered in
a goal for FC Duckburg . the stadium went wild as Nina Green was withdrawn on
the 82 th minute with her knee in a brace following a challenge from Terri Welch
. 7 minutes after that Nina Green scored on the 89 th minute drilling in the
ball in the right corner after she ran 13 metres and dribbled 10 metres on the
centre . the stadium went wild as Elsie Vajgrt fouled Laura Owens playing for FC
Duckburg for a interesting free-kick opportunity for FC Duckburg for which she
was booked .
Answer: Nina Green
Prediction: Nina Green
Question ID: 1f19b591e1cb356c60f34c08c1956e38
Question: When did they score the 1 th goal?
Passage: 
A spectacular 23 metres strike from Elsie 

In [17]:
print(ctr)
print(sum(1 for sample in gold for _ in sample_iter(sample)))

48
1074
